<a href="https://colab.research.google.com/github/patelshraddha/dsfg-2020/blob/master/Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Sources
- Existing Hubs - https://www.communityhubs.org.au/hubs/#hubs-anchor
- ACARA school profiles - https://www.acara.edu.au/contact-us/acara-data-access
- PHIDU Australian Social Atlas: Local Goverment Area - http://phidu.torrens.edu.au/social-health-atlases/data#social-health-atlases-of-australia-local-government-areas

# Methodology 

### Merge existing schools with Australian school profiles
Added the additional 13ish schools through fuzzy match - 70 schools total
Compare target (existing hubs) vs general population
- Bottom SEA Quarter, Top SEA Quarter
- Higher total enrolment
- Primary/Combined schools
- Higher language background other than english
- non teaching staff by enrollment

Feature engineering 
- Numerical variables
  - get student/ staff ratios
  - get girls/boys percentages by total
- Categorical variables 
  - 'Campus Type', 'Governing Body', 'Postcode', 'Rolled Reporting Description','School Sector', 'School Type', 'State', 'Suburb'
  - 'School Name' - different branches predictors - removed a bunch of stop words - public, east/west/north/south, primary/secondary/high
  - 'Year Range' - break it down to min (Prep, K, P, PP, R, T,U) to max (7)

### Get PHIDU socio-economic indicators for all school and hubs
- LGA code
- Families
- Migrant - Skilled, Humanitarian, Family
- Birthplace - NES 
- Early childhood Dev
- Child care
- Income Support

### Lookalike Modelling
- Get information value of each feature - to identify which features are the most responsible to make a school into a hub
- Manually select features for models - with and without any location data
- Create model
  - Similarity metric - Gower - categorical + numerical
  (https://pypi.org/project/gower/)
  - Select features to maximize recall - identifying all the existing hubs from the schools as quickly as possible
- Check if the model does well by comparing the ranked list and general population with the hubs



#Set up 
### Mount Google Drive
### Install libraries

In [2]:
# Optional - Mount if data in Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Get Google Maps API key for geocoding and reverse geocoding - https://developers.google.com/maps/documentation/javascript/get-api-key
GOOGLE_MAPS_API_KEY = ''

dir= ''
resultsdir= ''
codedir= ''
!ls ''

In [4]:
# Install all the required libraries
!apt-get update && apt-get install -y libspatialindex-dev
!pip install -U googlemaps
!pip install gower
!pip install Shapely==1.6.4.post2
!pip install descartes==1.1.0
!pip install geopandas==0.4.0
!pip install matplotlib==3.0.2
!pip install rtree==0.8.3
!pip install woe

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [92.1 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 

In [0]:
# Import libraries
import pandas as pd
import pandas.core.algorithms as algos
from pandas import Series
import os
import numpy as np
import gower
import geopandas as gpd
from shapely.geometry import Point
import googlemaps
from datetime import datetime
import math
import scipy.stats.stats as stats
import re
import traceback
import string
import woe

from woe.eval import plot_ks
import warnings
warnings.filterwarnings('ignore')
max_bin = 20
force_bin = 3

# Once off run - Match hubs with ACARA school profiles

In [0]:
# This needs to be done once - need not be done in subsequent runs!
# Read Hub locations - pair of latitude, longitude
xl_file = pd.ExcelFile(os.path.join(dir,'Hub locations.xlsx'))

hub_locations = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
df_hubs = hub_locations[list(hub_locations.keys())[0]]

# Read ACARA school profiles
xl_file = pd.ExcelFile(os.path.join(dir,'School Profile 2008-2018.xlsx'))

school_profile = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
school_profile = school_profile[list(school_profile.keys())[1]]

In [9]:
df_hubs.head()

,Hub ID,School Name,Location Coordinates
0,CH00084,Banksia Road Public School,-33.9068 | 151.05015
1,CH00001,Fernbrooke State School,-27.65852 | 152.83998
2,CH00003,Riverview State School,-27.6046 | 152.84662
3,CH00004,Staines Memorial College,-27.66902 | 152.84945
4,CH00005,Marsden State School,-27.68563 | 153.10479


In [0]:
## Snippet
import googlemaps
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)
# Reverse geocoding - get address in a JSON format from latitude, longitude pairs
dict_reverse_geocoded = {}
for index,row in df_hubs.iterrows():
  key = str(row['School Name'])+'|'+str(row['Latitude'])+'|'+str(row['Longitude'])
  if not math.isnan(row['Latitude']) and not math.isnan(row['Longitude']) :
    dict_reverse_geocoded[key] = gmaps.reverse_geocode((row['Latitude'], row['Longitude']))   

df_hubs['Postcode'] = df_hubs.apply(lambda row: get_address_field(row,'postal_code'),axis=1)

In [10]:
school_profile.head()

,Calendar Year,ACARA SML ID,AGE ID,School Name,Suburb,State,Postcode,School Sector,School Type,Campus Type,Rolled Reporting Description,School URL,Governing Body,Governing Body URL,Year Range,Geolocation,ICSEA,Bottom SEA Quarter (%),Lower Middle SEA Quarter (%),Upper Middle SEA Quarter (%),Top SEA Quarter (%),Teaching Staff,Full Time Equivalent Teaching Staff,Non-Teaching Staff,Full Time Equivalent Non-Teaching Staff,Total Enrolments,Girls Enrolments,Boys Enrolments,Full Time Equivalent Enrolments,Indigenous Enrolments (%),Language Background Other Than English (%)
0,2008,40000,3.0,Corpus Christi Catholic School,Bellerive,TAS,7018,Catholic,Primary,School Single Entity,Individual Reporting,http://www.corpuschristi.tas.edu.au,Tasmanian Catholic Education Office,http://www.catholic.tas.edu.au,"U, Prep-6",Inner Regional,1055.0,13.0,19.0,37.0,31.0,29.0,19.0,12.0,6.9,410.0,185.0,225.0,410.0,1.0,4.0
1,2009,40000,3.0,Corpus Christi Catholic School,Bellerive,TAS,7018,Catholic,Primary,School Single Entity,Individual Reporting,http://www.corpuschristi.tas.edu.au,Tasmanian Catholic Education Office,http://www.catholic.tas.edu.au,"U, Prep-6",Inner Regional,1055.0,13.0,19.0,37.0,31.0,27.0,19.1,15.0,8.7,409.0,184.0,225.0,409.0,2.0,4.0
2,2010,40000,3.0,Corpus Christi Catholic School,Bellerive,TAS,7018,Catholic,Primary,School Single Entity,Individual Reporting,http://www.corpuschristi.tas.edu.au,Tasmanian Catholic Education Office,http://www.catholic.tas.edu.au,"U, Prep-6",Inner Regional,1044.0,28.0,19.0,30.0,23.0,26.0,18.6,18.0,10.2,392.0,173.0,219.0,392.0,2.0,4.0
3,2011,40000,3.0,Corpus Christi Catholic School,Bellerive,TAS,7018,Catholic,Primary,School Single Entity,Individual Reporting,http://www.corpuschristi.tas.edu.au,Tasmanian Catholic Education Office,http://www.catholic.tas.edu.au,"U, Prep-6",Inner Regional,1046.0,19.0,30.0,25.0,25.0,25.0,18.1,16.0,9.0,385.0,173.0,212.0,385.0,0.0,3.0
4,2012,40000,3.0,Corpus Christi Catholic School,Bellerive,TAS,7018,Catholic,Primary,School Single Entity,Individual Reporting,http://www.corpuschristi.tas.edu.au,Tasmanian Catholic Education Office,http://www.catholic.tas.edu.au,"U, Prep-6",Inner Regional,1041.0,9.0,35.0,28.0,28.0,24.0,18.1,16.0,9.4,391.0,185.0,206.0,391.0,1.0,3.0


In [0]:
# function definitions
def get_address_field(row, field):
  # get specific address field eg. Postcode from address json
  key = str(row['School Name'])+'|'+str(row['Latitude'])+'|'+str(row['Longitude'])
  if not math.isnan(row['Latitude']) and not math.isnan(row['Longitude']):
    address_components = dict_reverse_geocoded[key][0]['address_components']
    for component in address_components:
      if field in component['types']:
        return int(component['long_name'])
  else:
    return None

def clean_school_name(name):
  # Removing common words from school names
  clean_name= name.lower().replace('primary','').replace('school','').replace('college','').replace('secondary','').replace('public','').replace('high','').replace('park','').replace('girls','').replace('boys','')
  clean_name= clean_name.replace('east','').replace('west','').replace('north','').replace('south','')
  clean_name=' '.join(clean_name.split())
  return clean_name.strip()

def get_lat_long(row, field):
  key = str(row['ACARA SML ID'])
  try:
    location = dict_geocoded[key][0]['geometry']['location']
    if field in location.keys():
      return location[field]
    else:
      return None
  except:
    return None

In [0]:
# Split location coordinates into latitude, longitude for hubs
df_hubs['Latitude'] = df_hubs['Location Coordinates'].apply(lambda x: float(str(x).split('|')[0]))
df_hubs['Longitude'] = df_hubs['Location Coordinates'].apply(lambda x: float(str(x).split('|')[1]) if '|' in str(x) else None)


gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)
# Reverse geocoding - get address in a JSON format from latitude, longitude pairs
dict_reverse_geocoded = {}
for index,row in df_hubs.iterrows():
  key = str(row['School Name'])+'|'+str(row['Latitude'])+'|'+str(row['Longitude'])
  if not math.isnan(row['Latitude']) and not math.isnan(row['Longitude']) :
    dict_reverse_geocoded[key] = gmaps.reverse_geocode((row['Latitude'], row['Longitude']))   

df_hubs['Postcode'] = df_hubs.apply(lambda row: get_address_field(row,'postal_code'),axis=1)

In [0]:
# Create additional features for schools
school_profile['Girls (%)'] = school_profile['Girls Enrolments'] / school_profile['Total Enrolments']
school_profile['Boys (%)'] = school_profile['Boys Enrolments'] / school_profile['Total Enrolments']
school_profile['Teaching Staff by enrolments'] = school_profile['Total Enrolments'] / school_profile['Teaching Staff']
school_profile['Non-Teaching Staff by enrolments'] = school_profile['Total Enrolments'] / school_profile['Non-Teaching Staff']

# Split Year Range into Start and End
school_profile['Year_Start'] = school_profile['Year Range'].apply(lambda x: str(x).split('-')[0])
school_profile['Year_End'] = school_profile['Year Range'].apply(lambda x: str(x).split('-')[1] if len(str(x).split('-')) > 1 else None)
school_profile['school_name_processed'] = school_profile['School Name'].apply(lambda x: clean_school_name(x))

# Get the latest school profile 
school_profile_filtered = school_profile.groupby(['ACARA SML ID'])['Calendar Year'].transform(max) == school_profile['Calendar Year'] 
school_profile_filtered = school_profile[school_profile_filtered]
school_profile_filtered = school_profile_filtered.replace([np.inf, -np.inf], np.nan)
school_profile_filtered = school_profile_filtered.fillna(0)

In [35]:
school_profile_filtered.head()

,Calendar Year,ACARA SML ID,AGE ID,School Name,Suburb,State,Postcode,School Sector,School Type,Campus Type,Rolled Reporting Description,School URL,Governing Body,Governing Body URL,Year Range,Geolocation,ICSEA,Bottom SEA Quarter (%),Lower Middle SEA Quarter (%),Upper Middle SEA Quarter (%),Top SEA Quarter (%),Teaching Staff,Full Time Equivalent Teaching Staff,Non-Teaching Staff,Full Time Equivalent Non-Teaching Staff,Total Enrolments,Girls Enrolments,Boys Enrolments,Full Time Equivalent Enrolments,Indigenous Enrolments (%),Language Background Other Than English (%),Girls (%),Boys (%),Teaching Staff by enrolments,Non-Teaching Staff by enrolments,Year_Start,Year_End,school_name_processed
10,2018,40000,3.0,Corpus Christi Catholic School,Bellerive,TAS,7018,Catholic,Primary,School Single Entity,Individual Reporting,http://www.corpuschristi.tas.edu.au,Tasmanian Catholic Education Office,http://www.catholic.tas.edu.au,Prep-6,Inner Regional,1080.0,6.0,25.0,33.0,36.0,29.0,20.8,18.0,10.3,380.0,179.0,201.0,380.0,2.0,3.0,0.471053,0.528947,13.103448,21.111111,Prep,6,corpus christi catholic
21,2018,40001,4.0,Fahan School,Sandy Bay,TAS,7005,Independent,Combined,School Single Entity,Individual Reporting,http://www.fahan.tas.edu.au,Independent Schools Tasmania,http://www.independentschools.tas.edu.au,Prep-12,Inner Regional,1135.0,3.0,10.0,26.0,61.0,41.0,35.0,27.0,19.0,390.0,390.0,0.0,390.0,1.0,7.0,1.000000,0.000000,9.512195,14.444444,Prep,12,fahan
32,2018,40002,5.0,Geneva Christian College,Latrobe,TAS,7307,Independent,Combined,School Single Entity,Individual Reporting,http://www.geneva.tas.edu.au,Independent Schools Tasmania,http://www.independentschools.tas.edu.au,Prep-12,Outer Regional,976.0,37.0,35.0,21.0,7.0,23.0,16.0,29.0,15.6,208.0,89.0,119.0,208.0,6.0,5.0,0.427885,0.572115,9.043478,7.172414,Prep,12,geneva christian
43,2018,40003,7.0,Holy Rosary Catholic School,Claremont,TAS,7011,Catholic,Primary,School Single Entity,Individual Reporting,http://www.holyrosary.tas.edu.au,Tasmanian Catholic Education Office,http://www.catholic.tas.edu.au,Prep-6,Inner Regional,981.0,33.0,40.0,22.0,5.0,28.0,23.5,24.0,11.3,399.0,176.0,223.0,399.0,5.0,1.0,0.441103,0.558897,14.250000,16.625000,Prep,6,holy rosary catholic
54,2018,40004,9.0,Immaculate Heart of Mary Catholic School,Lenah Valley,TAS,7008,Catholic,Primary,School Single Entity,Individual Reporting,http://www.ihms.tas.edu.au,Tasmanian Catholic Education Office,http://www.catholic.tas.edu.au,Prep-6,Inner Regional,1015.0,24.0,29.0,24.0,23.0,15.0,11.3,10.0,4.8,200.0,107.0,93.0,200.0,11.0,15.0,0.535000,0.465000,13.333333,20.000000,Prep,6,immaculate heart of mary catholic


In [0]:
%%time
dict_geocoded = {}
# Geocode all ACARA schools - get latitude, longitude from address
# This is done to identify the LGA that the school is in later 
for index,row in school_profile_filtered.iterrows():
  key= str(row['ACARA SML ID'])
  search_term = row['School Name']+ " "+row['Suburb']+ " "+row['State']+ " "+str(row['Postcode'])
  dict_geocoded[key] = gmaps.geocode(search_term)

CPU times: user 27.3 s, sys: 1.54 s, total: 28.8 s
Wall time: 57min 15s


In [0]:
# Create latitude, longitude columns for all schools
school_profile_filtered['School_Profile_Latitude'] = school_profile_filtered.apply(lambda row: get_lat_long(row,'lat'),axis=1)
school_profile_filtered['School_Profile_Longitude'] = school_profile_filtered.apply(lambda row: get_lat_long(row,'lng'),axis=1)

In [0]:
# Merge with hubs to get additional information - enrollment etc. 
df_merged = df_hubs.merge(school_profile_filtered, how='inner')

In [0]:
# Some hubs might be missed with this merge
# The following is a method to add them
'''
# To get the missing schools
set(df_hubs['School Name']) - set(df_merged['School Name'])

# Create some search terms from the list for fuzzy match
search_terms = ['Blair', 'Gowrie', 'Holy Child', 'Northern Bay', 'Paralowie','St Anthony','St Dominic', 'St Francis','St Paul','Wilmot','St Georges','Woodlink','Wyndham']

# To get the hubs data for missing records
df_hubs[df_hubs['School Name'].str.contains('|'.join(search_terms),na=False)].sort_values('School Name')

# To get similar schools based on the search terms
for term in search_terms:
  with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(school_profile_filtered[school_profile_filtered['School Name'].str.contains(term,na=False)][['School Name','ACARA SML ID','AGE ID','Postcode']])

# Manually note down the hub id and corresponding ACARA SML ID and AGE ID
ACARA SML ID AGE ID
{'Blair Athol North School' Blair Athol North School B-7 - 50339	7040.0,
 'Gowrie Street Primary School' Gowrie Street Primary School Shepparton - 44773	14790.0, Postcode is 3630 in API and 3632 in the larger schools dataset
 'Holy Child Primary School', Holy Child School, 45978	801.0	
 'Northern Bay College',Northern Bay P-12 College         50291  29152.0
 'Paralowie R-12 School', Paralowie School         49597  6083.0
 "St Anthony's Primary School", St Anthony's School         45895    965.0
 "St Dominic's Primary School", St Dominic's School         45937  1009.0  
 'St Francis College', St Francis' College         47772    5609.0    
 'St Georges Road Primary School', St Georges Road Primary School Shepparton         44778  11471.0
 "St Paul's Primary School", St Paul's School         47729    687.0 
 'Wilmot Road Primary School', Wilmot Road Primary School Shepparton         44927  11468.0  
 'Woodlinks State School', WoodLinks State School         50366  68203.0  
 'Wyndham Park Primary', Wyndham Park Primary School         50303  29139.0
 }
'''

In [0]:
# Adding schools missed by the merge
dict_missing_hubs = {'CH00070': 50339,
 'CH00062':44773,
 'CH00052':45978,
 'CH00061':50291,
 'CH00065':49597,
 'CH00044':45895,
 'CH00058':45937,
 'CH00007':47772,
 'CH00063':44778,
 'CH00008':47729,
 'CH00064':44927,
 'CH00006':50366,
 'CH00086':50303}
missing_df = pd.DataFrame(dict_missing_hubs.items(), columns=['Hub ID', 'ACARA SML ID'])
df_missing_merged = df_hubs[['Hub ID','Location Coordinates','Latitude','Longitude']].merge(missing_df, how='inner').merge(school_profile_filtered,how='inner')
df_merged = pd.concat([df_merged,df_missing_merged[df_merged.columns]])

In [0]:
# Write to csv so that we do not need to run the geocoding and reverse geocoding steps again. (Google Maps API is expensive)
df_merged.to_csv(os.path.join(resultsdir,'hubs_merged_geocoded.csv'),header=True, index=False)
school_profile_filtered.to_csv(os.path.join(resultsdir,'school_profile_geocoded.csv'),header=True, index=False)

# Process Phidu Datasets

These datasets are added to get the socio-economic indicators for the Local Government Areas (LGA) that the schools/hubs reside in.

In [0]:
# Phidu datasets have been cleaned manually - remove dirty rows, convert to numbers etc
df_phidu_families = pd.read_csv(os.path.join(codedir, 'Phidu_Families.csv'))
df_phidu_families['LGA region']= df_phidu_families['Name'].apply(lambda x: str(x).split('(')[0].strip().lower())

df_phidu_early_childhood = pd.read_csv(os.path.join(codedir, 'Phidu_Early_Childhood_Dev.csv'))
df_phidu_early_childhood['LGA region']= df_phidu_early_childhood['Name'].apply(lambda x: str(x).split('(')[0].strip().lower())

df_phidu_birthplace = pd.read_csv(os.path.join(codedir, 'Phidu_Birthplace.csv'))
df_phidu_birthplace['LGA region']= df_phidu_birthplace['Name'].apply(lambda x: str(x).split('(')[0].strip().lower())

df_phidu_birthplace = pd.read_csv(os.path.join(codedir, 'Phidu_Birthplace.csv'))
df_phidu_birthplace['LGA region']= df_phidu_birthplace['Name'].apply(lambda x: str(x).split('(')[0].strip().lower())

df_phidu_migrant_humanitarian = pd.read_csv(os.path.join(codedir, 'Phidu_Migrant_Humanitarian.csv'))
df_phidu_migrant_humanitarian['LGA region']= df_phidu_migrant_humanitarian['Name'].apply(lambda x: str(x).split('(')[0].strip().lower())

df_phidu_migrant_skilled = pd.read_csv(os.path.join(codedir, 'Phidu_Migrant_Skilled.csv'))
df_phidu_migrant_skilled['LGA region']= df_phidu_migrant_skilled['Name'].apply(lambda x: str(x).split('(')[0].strip().lower())

df_phidu_migrant_family = pd.read_csv(os.path.join(codedir, 'Phidu_Migrant_Family.csv'))
df_phidu_migrant_family['LGA region']= df_phidu_migrant_family['Name'].apply(lambda x: str(x).split('(')[0].strip().lower())

df_phidu_income_support = pd.read_csv(os.path.join(codedir, 'Phidu_Income_Support.csv'))
df_phidu_income_support['LGA region']= df_phidu_income_support['Name'].apply(lambda x: str(x).split('(')[0].strip().lower())

df_phidu_child_care = pd.read_csv(os.path.join(codedir, 'Phidu_Child_Care.csv'))
df_phidu_child_care['LGA region']= df_phidu_child_care['Name'].apply(lambda x: str(x).split('(')[0].strip().lower())

In [0]:
# Merge the PHIDU datasets by LGA
df_phidu= None
for df in [df_phidu_families, df_phidu_early_childhood, df_phidu_birthplace, df_phidu_migrant_skilled, 
           df_phidu_migrant_humanitarian, df_phidu_migrant_family, df_phidu_income_support, df_phidu_child_care]:
  if df_phidu is not None:
    df_phidu= df_phidu.merge(df)
  else:
    df_phidu= df

In [36]:
df_phidu.head()

,Code,Name,Single parent families with children under 15 years,Total families with children under 15 years,% single parent families,Jobless familes with children under 15 years,Total families with children under 15 years.1,% jobless families,Children under 15 years in jobless families,Total children under 15 years,% children in jobless families,Children aged less than 15 years living in families where the female parent’s highest level of schooling was year 10 or below/female parent did not attend school,Children aged less than 15 years,% children in families where the mother has low educational attainment,LGA region,Children developmentally vulnerable on one or more domains,Children assessed in AEDC (first year of school),% Children developmentally vulnerable on one or more domains,Children developmentally vulnerable on two or more domains,Children assessed in AEDC (first year of school).1,% Children developmentally vulnerable on two or more domains,Children developmentally vulnerable in physical domain,Physical domain - valid scores,% Children developmentally vulnerable in physical domain,Children developmentally at risk in physical domain,Physical domain - valid scores.1,% Children developmentally at risk in physical domain,Children developmentally on track in physical domain,Physical domain - valid scores.2,% Children developmentally on track in physical domain,Children developmentally vulnerable in social domain,Social domain - valid scores,% Children developmentally vulnerable in social domain,Children developmentally at risk in social domain,Social domain - valid scores.1,% Children developmentally at risk in social domain,Children developmentally on track in social domain,Social domain - valid scores.2,% Children developmentally on track in social domain,Children developmentally vulnerable in emotional domain,...,% disability support pensioners,Female sole parent pensioners,Females aged 15 to 54 years,% female sole parent pensioners,People receiving an unemployment benefit,Persons aged 16 to 64 years.1,% people receiving an unemployment benefit,People receiving an unemployment benefit for less than 6 months,Persons aged 16 to 64 years.2,% people receiving an unemployment benefit short-term,People receiving an unemployment benefit for longer than 6 months,Persons aged 16 to 64 years.3,% people receiving an unemployment benefit long-term,Young people \n(16 to 24 years) receiving an unemployment benefit,Persons aged 16 to 24 years,% young people receiving an unemployment benefit,"Low income, welfare-dependent families (with children)",Total families,"% low income, welfare-dependent families (with children)","Children in low income, welfare-dependent families",Children under 16 years,"% children in low income, welfare-dependent families",Health Care Card holders,Persons 0 to 64 years,% Health Care Card holders,Pensioner Concession Card holders,Persons aged 15 years and over,% Pensioner Concession Card holders,Seniors Health Card holders,Persons aged 65 years and over.1,% Seniors Health Card holders,People aged 15 years and over providing child care to their own child/ children,Population aged 15 years and over,% providing child care to their own child/ children,People aged 15 years and over providing child care to other child/ children,Population aged 15 years and over.1,% providing child care to other child/ children,People aged 15 years and over providing child care,Population aged 15 years and over.2,% providing child care
0,10050,Albury (C),1317,5023,26.219391,726,5023,14.453514,1287,9579,13.435640,1995,9579,20.826809,albury,119.000000,589.000000,20.203735,59.000000,591.000000,9.983080,46.000000,591.000000,7.783418,56.724292,592.783109,9.569148,489.000000,591.000000,82.741117,62.000000,591.000000,10.490694,87.000000,591.000000,14.720812,442.000000,591.000000,74.788494,49.000000,...,7.573335,805.0,13672.666440,5.887659,2607.0,32759.677840,7.957954,421.0,32759.67784,1.285116,2186.0,32759.677840,6.672837,437.0,6649.669129,

In [0]:
# Get geographic information about the LGA in Phidu datasets to identify the socio-economic indicators for the LGA that the schools/hubs reside in
# Download and process LGA geoJSON file
# https://data.gov.au/search?q=local%20government%20area%20PSMA
gdf_lga_tas = gpd.read_file('https://data.gov.au/geoserver/as-local-government-areas-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_02565094_6b52_4d51_8a35_2273c711d60d&outputFormat=json')
gdf_lga_wa = gpd.read_file('https://data.gov.au/geoserver/wa-local-government-areas-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_8a8e0037_e474_422f_8026_241c7c88551f&outputFormat=json')
gdf_lga_sa = gpd.read_file('https://data.gov.au/geoserver/sa-local-government-areas-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_b71223e0_2235_4973_90e9_d279687b7ef8&outputFormat=json')
gdf_lga_vic = gpd.read_file('https://data.gov.au/geoserver/vic-local-government-areas-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_bdf92691_c6fe_42b9_a0e2_a4cd716fa811&outputFormat=json')
gdf_lga_nsw = gpd.read_file('https://data.gov.au/geoserver/nsw-local-government-areas/wfs?request=GetFeature&typeName=ckan_f6a00643_1842_48cd_9c2f_df23a3a1dc1e&outputFormat=json')
gdf_lga_nt = gpd.read_file('https://data.gov.au/geoserver/nt-local-government-areas-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_225a1af2_9b93_4162_9fe6_11e165255942&outputFormat=json')
gdf_lga_qld = gpd.read_file('https://data.gov.au/geoserver/qld-local-government-areas-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_16803f0b_6934_41ae_bf82_d16265784c7f&outputFormat=json')
gdf_lga_other = gpd.read_file('https://data.gov.au/geoserver/other-local-government-areas-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_a32e052d_1031_4840_bf22_859527c08024&outputFormat=json')

In [0]:
df_lga = gpd.GeoDataFrame()
dict_lga = {
    'TAS':{'df': gdf_lga_tas, 'full_name_col': 'tas_lga__2', 'name_col': 'tas_lga__3'},
    'WA':{'df': gdf_lga_wa, 'full_name_col': 'wa_lga_s_2', 'name_col': 'wa_lga_s_3'},
    'SA':{'df': gdf_lga_sa, 'full_name_col': 'sa_lga_s_2', 'name_col': 'sa_lga_s_3'},
    'VIC':{'df': gdf_lga_vic, 'full_name_col': 'vic_lga__2', 'name_col': 'vic_lga__3'},
    'NSW':{'df': gdf_lga_nsw, 'full_name_col': 'nsw_lga__2', 'name_col': 'nsw_lga__3'},
    'NT':{'df': gdf_lga_nt, 'full_name_col': 'nt_lga_s_2', 'name_col': 'nt_lga_s_3'},
    'QLD':{'df': gdf_lga_qld, 'full_name_col': 'qld_lga__2', 'name_col': 'qld_lga__3'},
    'OTH':{'df': gdf_lga_other, 'full_name_col': 'ot_lga_s_2', 'name_col': 'ot_lga_s_3'},
}

# get LGA names and geometries
for key in dict_lga.keys():
  df_test= dict_lga[key]['df'][['geometry',dict_lga[key]['full_name_col'],dict_lga[key]['name_col']]]
  df_test['state'] = key
  df_test = df_test.rename(columns={dict_lga[key]['full_name_col']: 'lga_name', dict_lga[key]['name_col']: 'name'})
  df_lga= df_lga.append(df_test)
df_lga['name'] = df_lga['name'].apply(lambda x: str(x).lower())

In [0]:
# Merge with Phidu datasets
df_phidu= df_lga.merge(df_phidu, right_on='LGA region', left_on='name')

In [42]:
df_phidu.head()

,geometry,lga_name,name,state,Code,Name,Single parent families with children under 15 years,Total families with children under 15 years,% single parent families,Jobless familes with children under 15 years,Total families with children under 15 years.1,% jobless families,Children under 15 years in jobless families,Total children under 15 years,% children in jobless families,Children aged less than 15 years living in families where the female parent’s highest level of schooling was year 10 or below/female parent did not attend school,Children aged less than 15 years,% children in families where the mother has low educational attainment,LGA region,Children developmentally vulnerable on one or more domains,Children assessed in AEDC (first year of school),% Children developmentally vulnerable on one or more domains,Children developmentally vulnerable on two or more domains,Children assessed in AEDC (first year of school).1,% Children developmentally vulnerable on two or more domains,Children developmentally vulnerable in physical domain,Physical domain - valid scores,% Children developmentally vulnerable in physical domain,Children developmentally at risk in physical domain,Physical domain - valid scores.1,% Children developmentally at risk in physical domain,Children developmentally on track in physical domain,Physical domain - valid scores.2,% Children developmentally on track in physical domain,Children developmentally vulnerable in social domain,Social domain - valid scores,% Children developmentally vulnerable in social domain,Children developmentally at risk in social domain,Social domain - valid scores.1,% Children developmentally at risk in social domain,...,% disability support pensioners,Female sole parent pensioners,Females aged 15 to 54 years,% female sole parent pensioners,People receiving an unemployment benefit,Persons aged 16 to 64 years.1,% people receiving an unemployment benefit,People receiving an unemployment benefit for less than 6 months,Persons aged 16 to 64 years.2,% people receiving an unemployment benefit short-term,People receiving an unemployment benefit for longer than 6 months,Persons aged 16 to 64 years.3,% people receiving an unemployment benefit long-term,Young people \n(16 to 24 years) receiving an unemployment benefit,Persons aged 16 to 24 years,% young people receiving an unemployment benefit,"Low income, welfare-dependent families (with children)",Total families,"% low income, welfare-dependent families (with children)","Children in low income, welfare-dependent families",Children under 16 years,"% children in low income, welfare-dependent families",Health Care Card holders,Persons 0 to 64 years,% Health Care Card holders,Pensioner Concession Card holders,Persons aged 15 years and over,% Pensioner Concession Card holders,Seniors Health Card holders,Persons aged 65 years and over.1,% Seniors Health Card holders,People aged 15 years and over providing child care to their own child/ children,Population aged 15 years and over,% providing child care to their own child/ children,People aged 15 years and over providing child care to other child/ children,Population aged 15 years and over.1,% providing child care to other child/ children,People aged 15 years and over providing child care,Population aged 15 years and over.2,% providing child care
0,"(POLYGON ((147.22215845 -43.31537873, 147.2223...",KINGBOROUGH COUNCIL,kingborough,TAS,63610,Kingborough (M),824,3859,21.352682,337,3862,8.726049,581,7127,8.152098,1150,7127,16.135822,kingborough,66.0,455.0,14.505495,21.0,455.0,4.615385,30.0,455.0,6.593407,41.041416,454.060931,9.038746,384.0,455.0,84.395604,16.0,455.0,3.516484,53.0,455.0,11.648352,...,6.040858,319.0,8944.433579,3.566464,968.0,22248.49487,4.350856,158.0,22248.49487,0.71016,810.0,22248.49487,3.640696,121.0,3586.799007,3.373481,705.0,9979,7.064836,1274.0,7886.476658,16.154235,2250.0,30134.97152,7.466408,6754.0,29681.99558,22.754535,763.0,6965.026951,10.954731,6200,28724,21.584737,2509,28724,8.7

# Merge PHIDU Schools/Hubs information

In [0]:
# Read previously saved hubs and schools datasets 
df_merged= pd.read_csv(os.path.join(resultsdir,'hubs_merged_geocoded.csv'))
school_profile_filtered= pd.read_csv(os.path.join(resultsdir,'school_profile_geocoded.csv'))

In [0]:
# Merge hubs with Phidu data - match which schools/hubs exist "within" LGA 
crs = {'init': 'epsg:4326'}
geometry = [Point(xy) for xy in zip(df_merged.Longitude, df_merged.Latitude)]
gdf_hubs = gpd.GeoDataFrame(df_merged, crs=crs, geometry=geometry)
df_merged = gpd.sjoin(gdf_hubs, df_phidu, op="within")

In [0]:
# Merge schools with Phidu data - match which schools/hubs exist "within" LGA 
geometry = [Point(xy) for xy in zip(school_profile_filtered.School_Profile_Longitude, school_profile_filtered.School_Profile_Latitude)]
gdf_schools = gpd.GeoDataFrame(school_profile_filtered, crs=crs, geometry=geometry)
school_profile_filtered = gpd.sjoin(gdf_schools, df_phidu, op="within")

In [49]:
# Distribution of numerical columns for all schools
school_profile_filtered.describe()

,Calendar Year,ACARA SML ID,AGE ID,Postcode,ICSEA,Bottom SEA Quarter (%),Lower Middle SEA Quarter (%),Upper Middle SEA Quarter (%),Top SEA Quarter (%),Teaching Staff,Full Time Equivalent Teaching Staff,Non-Teaching Staff,Full Time Equivalent Non-Teaching Staff,Total Enrolments,Girls Enrolments,Boys Enrolments,Full Time Equivalent Enrolments,Indigenous Enrolments (%),Language Background Other Than English (%),Girls (%),Boys (%),Teaching Staff by enrolments,Non-Teaching Staff by enrolments,School_Profile_Latitude,School_Profile_Longitude,index_right,Code,Single parent families with children under 15 years,Total families with children under 15 years,% single parent families,Jobless familes with children under 15 years,Total families with children under 15 years.1,% jobless families,Children under 15 years in jobless families,Total children under 15 years,% children in jobless families,Children aged less than 15 years living in families where the female parent’s highest level of schooling was year 10 or below/female parent did not attend school,Children aged less than 15 years,% children in families where the mother has low educational attainment,Children developmentally vulnerable on one or more domains,...,% disability support pensioners,Female sole parent pensioners,Females aged 15 to 54 years,% female sole parent pensioners,People receiving an unemployment benefit,Persons aged 16 to 64 years.1,% people receiving an unemployment benefit,People receiving an unemployment benefit for less than 6 months,Persons aged 16 to 64 years.2,% people receiving an unemployment benefit short-term,People receiving an unemployment benefit for longer than 6 months,Persons aged 16 to 64 years.3,% people receiving an unemployment benefit long-term,Young people \n(16 to 24 years) receiving an unemployment benefit,Persons aged 16 to 24 years,% young people receiving an unemployment benefit,"Low income, welfare-dependent families (with children)",Total families,"% low income, welfare-dependent families (with children)","Children in low income, welfare-dependent families",Children under 16 years,"% children in low income, welfare-dependent families",Health Care Card holders,Persons 0 to 64 years,% Health Care Card holders,Pensioner Concession Card holders,Persons aged 15 years and over,% Pensioner Concession Card holders,Seniors Health Card holders,Persons aged 65 years and over.1,% Seniors Health Card holders,People aged 15 years and over providing child care to their own child/ children,Population aged 15 years and over,% providing child care to their own child/ children,People aged 15 years and over providing child care to other child/ children,Population aged 15 years and over.1,% providing child care to other child/ children,People aged 15 years and over providing child care,Population aged 15 years and over.2,% providing child care
count,9413.0,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,...,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9.413000e+03,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000
mean,2018.0,45453.645384,16620.012111,3705.863593,977.210347,30.016467,25.144481,22.840221,19.800382,33.049931,30.491884,17.878466,12.708871,414.673749,202.762244,211.90

In [0]:
# Distribution of numerical columns for hubs
df_merged.describe()

,Latitude,Longitude,Postcode,Calendar Year,ACARA SML ID,AGE ID,ICSEA,Bottom SEA Quarter (%),Lower Middle SEA Quarter (%),Upper Middle SEA Quarter (%),Top SEA Quarter (%),Teaching Staff,Full Time Equivalent Teaching Staff,Non-Teaching Staff,Full Time Equivalent Non-Teaching Staff,Total Enrolments,Girls Enrolments,Boys Enrolments,Full Time Equivalent Enrolments,Indigenous Enrolments (%),Language Background Other Than English (%),Girls (%),Boys (%),Teaching Staff by enrolments,Non-Teaching Staff by enrolments,Year_End,School_Profile_Latitude,School_Profile_Longitude,index_right,Code,Single parent families with children under 15 years,Total families with children under 15 years,% single parent families,Jobless familes with children under 15 years,Total families with children under 15 years.1,% jobless families,Children under 15 years in jobless families,Total children under 15 years,% children in jobless families,Children aged less than 15 years living in families where the female parent’s highest level of schooling was year 10 or below/female parent did not attend school,...,% disability support pensioners,Female sole parent pensioners,Females aged 15 to 54 years,% female sole parent pensioners,People receiving an unemployment benefit,Persons aged 16 to 64 years.1,% people receiving an unemployment benefit,People receiving an unemployment benefit for less than 6 months,Persons aged 16 to 64 years.2,% people receiving an unemployment benefit short-term,People receiving an unemployment benefit for longer than 6 months,Persons aged 16 to 64 years.3,% people receiving an unemployment benefit long-term,Young people \n(16 to 24 years) receiving an unemployment benefit,Persons aged 16 to 24 years,% young people receiving an unemployment benefit,"Low income, welfare-dependent families (with children)",Total families,"% low income, welfare-dependent families (with children)","Children in low income, welfare-dependent families",Children under 16 years,"% children in low income, welfare-dependent families",Health Care Card holders,Persons 0 to 64 years,% Health Care Card holders,Pensioner Concession Card holders,Persons aged 15 years and over,% Pensioner Concession Card holders,Seniors Health Card holders,Persons aged 65 years and over.1,% Seniors Health Card holders,People aged 15 years and over providing child care to their own child/ children,Population aged 15 years and over,% providing child care to their own child/ children,People aged 15 years and over providing child care to other child/ children,Population aged 15 years and over.1,% providing child care to other child/ children,People aged 15 years and over providing child care,Population aged 15 years and over.2,% providing child care
count,70.000000,70.000000,70.000000,70.0,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,...,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,-34.851820,147.318294,3177.885714,2018.0,45088.314286,14992.185714,938.342857,55.371429,24.114286,14.071429,6.485714,39.828571,38.392857,19.842857,14.370000,522.928571,252.014286,270.914286,522.608571,5.042857,69.657143,0.482772,0.517228,13.327641,45.000472,6.514286,-34.851914,147.318904,1134.900000,23672.000000,4864.371429,23170.157143,21.368752,3988.500000,23169.842857,17.587349,7568.300000,44059.814286,17.521059,9187.542857,...,5.917488,2651.121782,6252

# Lookalike Modelling
### Two models were created:
#### ACARA_PHIDU_location: Model with all features
#### ACARA_PHIDU_no_location: No LGA/school related information



In [0]:
school_profile_test = school_profile_filtered.copy()
df_merged_test = df_merged.copy()
school_profile_test.drop('geometry', inplace=True, axis=1)
df_merged_test.drop('geometry', inplace=True, axis=1)
# Convert LGA codes to string to get LGA codes where the Community Hubs already exist
# This is done to atleast include LGA(s) already considered and liked
school_profile_test['Code'] = school_profile_test['Code'].astype(str)
df_merged_test['Code'] = df_merged_test['Code'].astype(str)
# Prevent issues if hubs and schools are read from the previously saved csv rather than being generated in the notebook
df_merged_test['Postcode'] = df_merged_test['Postcode'].astype(int)
df_merged_test['Year_End'] = df_merged_test['Year_End'].astype(str)

# Create models
dict_models ={ 'ACARA_PHIDU_location' :{
                  'numerical_features': ['ICSEA','Bottom SEA Quarter (%)', 'Lower Middle SEA Quarter (%)','Upper Middle SEA Quarter (%)', 
                      'Top SEA Quarter (%)', 'Teaching Staff',
                      'Full Time Equivalent Teaching Staff', 'Non-Teaching Staff',
                      'Full Time Equivalent Non-Teaching Staff', 'Total Enrolments',
                      'Girls Enrolments', 'Boys Enrolments',
                      'Full Time Equivalent Enrolments', 'Indigenous Enrolments (%)',
                      'Language Background Other Than English (%)', 'Girls (%)', 'Boys (%)',
                      'Teaching Staff by enrolments', 'Non-Teaching Staff by enrolments'],
                    'categorical_features': ['Campus Type','Rolled Reporting Description','School Sector', 
                        'School Type', 'Governing Body','Year_Start','Year_End','school_name_processed','Geolocation',
                        'Code','State', 'Suburb', 'Postcode'],
                    'phidu_features' : [col for col in df_phidu.columns if col not in ['geometry', 'lga_name', 'name', 'state','Name','LGA region','Code']]
                      },
               'ACARA_PHIDU_no_location' : {
                   'numerical_features': ['ICSEA','Bottom SEA Quarter (%)', 'Lower Middle SEA Quarter (%)','Upper Middle SEA Quarter (%)', 
                      'Top SEA Quarter (%)', 'Teaching Staff',
                      'Full Time Equivalent Teaching Staff', 'Non-Teaching Staff',
                      'Full Time Equivalent Non-Teaching Staff', 'Total Enrolments',
                      'Girls Enrolments', 'Boys Enrolments',
                      'Full Time Equivalent Enrolments', 'Indigenous Enrolments (%)',
                      'Language Background Other Than English (%)', 'Girls (%)', 'Boys (%)',
                      'Teaching Staff by enrolments', 'Non-Teaching Staff by enrolments'],
                    'categorical_features': ['Campus Type','Rolled Reporting Description','School Sector', 
                        'School Type', 'Governing Body','Year_Start','Year_End','Geolocation'],
                    'phidu_features' : [col for col in df_phidu.columns if col not in ['geometry', 'lga_name', 'name', 'state','Name','LGA region','Code']]
               }
             }

In [0]:
# Calculate Weight of evidence and feature importance
# Reference: https://www.kaggle.com/pavansanagapati/weight-of-evidence-woe-information-value-iv
def mono_bin(Y, X, n = max_bin):
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [52]:
# Get information value for all the features w.r.t. target - is the school a hub or not?
df_test= school_profile_test.copy()
df_test['target'] = school_profile_test['ACARA SML ID'].apply(lambda x: 1 if x in list(df_merged_test['ACARA SML ID']) else 0)
final_iv, IV = data_vars(df_test,df_test.target)
IV.sort_values('IV',ascending=False).head(50)

,VAR_NAME,IV
171,Suburb,3.556326
191,lga_name,2.310415
85,Code,2.310415
192,name,2.310415
107,LGA region,2.310415
114,Name,2.310415
193,school_name_processed,2.111986
11,% Children developmentally vulnerable in commu...,1.987349
39,% children in jobless families,1.851551
56,Bottom SEA Quarter (%),1.567436


In [56]:
for model in dict_models.keys():
  recall= 0 
  precision= 0
  #features= [feature for (feature, recall) in sorted(dict_models[model]['feature_impact'].items(),key=lambda kv: kv[1], reverse=True)]
  model_features= dict_models[model]['numerical_features'] + dict_models[model]['categorical_features']+ dict_models[model]['phidu_features']
  # Rearrange features to select in the descending order of feature importance
  features = [feature for feature in list(IV.sort_values('IV',ascending=False)['VAR_NAME']) if feature in model_features]
  # Choosing K such that there is ample opportunity to capture all the existing hubs
  K = math.ceil(len(df_merged)*1.5)
  selected_features= []
  for feature in features:
    test_features= selected_features + [feature]
    # Calculate and sum distances of all schools from current hubs
    mat_distance= gower.gower_matrix(np.asarray(df_merged_test[test_features]),np.asarray(school_profile_test[test_features]))
    mat_ranked = np.sum(np.square(mat_distance),axis=0)
    school_profile_ranked = school_profile_test.copy()
    school_profile_ranked['Distance'] = mat_ranked
    # Sort by distance and rank results
    school_profile_ranked= school_profile_ranked.sort_values('Distance', ascending=True)
    school_profile_ranked = school_profile_ranked.reset_index()
    school_profile_ranked['Rank'] = school_profile_ranked.index + 1
    # Get the ranks and distances for current hubs
    df_hubs_ranked = df_merged_test.merge(school_profile_ranked, how='inner')

    # Get the recall and precision for the topK results
    df_topK = df_hubs_ranked[df_hubs_ranked['Rank'] <= K]
    test_recall = len(df_topK)/len(df_hubs_ranked)
    test_precision = len(df_topK)/K

    # Select feature if recall is more than the current value - if the new model is able to pick more hubs in the first K spots than the prevoous model
    if test_recall > recall:
      selected_features.append(feature)
      recall= test_recall
      precision= test_precision

  # Create final model with the selected features
  mat_distance= gower.gower_matrix(np.asarray(df_merged_test[selected_features]),np.asarray(school_profile_test[selected_features]))
  mat_ranked = np.sum(np.square(mat_distance),axis=0)
  school_profile_ranked = school_profile_test.copy()
  school_profile_ranked['Distance'] = mat_ranked
  school_profile_ranked= school_profile_ranked.sort_values('Distance', ascending=True)
  school_profile_ranked = school_profile_ranked.reset_index()
  school_profile_ranked['Rank'] = school_profile_ranked.index + 1

  # Merge with the hubs to analyze the ranks that the model gives for the existing hubs
  df_hubs_ranked = df_merged_test.merge(school_profile_ranked, how='inner')
  # Save schools and hubs ranked
  school_profile_ranked.to_csv(os.path.join(resultsdir,'schools_ranked_'+model+'.csv'),header=True, index=False)
  df_hubs_ranked.to_csv(os.path.join(resultsdir,'hubs_compare_'+model+'.csv'),header=True, index=False)
  dict_models[model]['school_profile_ranked'] = school_profile_ranked
  dict_models[model]['hubs_profile_ranked'] = df_hubs_ranked
  dict_models[model]['selected_features'] = selected_features
  print(model,selected_features, recall, precision)

ACARA_PHIDU_location ['Suburb', 'school_name_processed', '% Children developmentally vulnerable in communication domain'] 0.6571428571428571 0.4380952380952381
ACARA_PHIDU_no_location ['% children in jobless families', 'Language Background Other Than English (%)', '% Permanent migrants under the Humanitarian Program\n(2000 to 2006)', 'Pensioner Concession Card holders', 'Health Care Card holders', 'Jobless familes with children under 15 years', 'Children developmentally vulnerable in language and cognitive domain', 'People receiving an unemployment benefit', 'People receiving an unemployment benefit for less than 6 months', 'Year_Start'] 0.3 0.2


# How well does the model do?
- Summarize selected columns for all schools vs top ranked K schools
- Check if the top K ranked schools are similar to the hubs

In [57]:
model= 'ACARA_PHIDU_no_location'
# Summarize all schools
dict_models[model]['school_profile_ranked'][dict_models[model]['selected_features']].describe()

,% children in jobless families,Language Background Other Than English (%),% Permanent migrants under the Humanitarian Program\n(2000 to 2006),Pensioner Concession Card holders,Health Care Card holders,Jobless familes with children under 15 years,Children developmentally vulnerable in language and cognitive domain,People receiving an unemployment benefit,People receiving an unemployment benefit for less than 6 months
count,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000,9413.000000
mean,12.110354,22.974716,0.247445,23685.982368,9267.756110,1673.956974,109.033618,4863.444054,860.583667
std,6.247249,26.389044,0.423950,27867.838599,12195.576842,2058.780574,150.251945,6230.812966,1197.092313
min,0.000000,0.000000,0.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.680030,4.000000,0.000000,5674.000000,1835.000000,331.000000,20.000000,1026.000000,157.000000
50%,11.665732,11.000000,0.075706,14716.391900,5578.000000,902.000000,51.000000,2689.000000,447.000000
75%,14.482853,34.000000,0.272067,29845.000000,11305.000000,2146.000000,127.000000,6383.000000,1070.000000
max,70.600858,100.000000,2.510210,133056.000000,62089.000000,9049.000000,698.000000,30678.000000,5969.000000


In [58]:
# Summarize the top ranked K schools
dict_models[model]['school_profile_ranked'][dict_models[model]['selected_features']].iloc[0:K].describe()

,% children in jobless families,Language Background Other Than English (%),% Permanent migrants under the Humanitarian Program\n(2000 to 2006),Pensioner Concession Card holders,Health Care Card holders,Jobless familes with children under 15 years,Children developmentally vulnerable in language and cognitive domain,People receiving an unemployment benefit,People receiving an unemployment benefit for less than 6 months
count,105.000000,105.000000,105.000000,105.000000,105.000000,105.000000,105.000000,105.00000,105.000000
mean,17.130081,73.609524,1.215008,35972.714286,19368.952381,3658.561905,254.200000,9582.52381,1519.638095
std,2.609262,13.436815,0.310920,4413.169750,1929.882940,448.600965,49.802379,1595.20516,255.759883
min,11.401050,39.000000,0.576991,24260.000000,15093.000000,2947.000000,167.000000,6905.00000,1116.000000
25%,18.294177,67.000000,1.163832,33866.000000,18167.000000,3145.000000,237.000000,7409.00000,1271.000000
50%,18.294177,77.000000,1.163832,36923.000000,19907.000000,3722.000000,263.000000,10517.00000,1514.000000
75%,18.593661,84.000000,1.499169,39254.000000,21211.000000,4170.000000,304.000000,10673.00000,1841.000000
max,19.176743,97.000000,1.702247,39254.000000,21211.000000,4170.000000,304.000000,10673.00000,1841.000000


In [59]:
# Summarize the hubs
dict_models[model]['hubs_profile_ranked'][dict_models[model]['selected_features']].describe()

,% children in jobless families,Language Background Other Than English (%),% Permanent migrants under the Humanitarian Program\n(2000 to 2006),Pensioner Concession Card holders,Health Care Card holders,Jobless familes with children under 15 years,Children developmentally vulnerable in language and cognitive domain,People receiving an unemployment benefit,People receiving an unemployment benefit for less than 6 months
count,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,17.521059,69.657143,1.177319,40574.426185,18358.933129,3988.500000,240.345228,9708.572045,1584.637387
std,3.620733,23.341749,0.608028,12514.103199,5242.746633,1340.508542,111.902289,2733.557153,539.583852
min,7.541224,8.000000,0.145614,14620.000000,5578.000000,1100.000000,92.000000,3126.000000,475.000000
25%,16.139400,52.500000,0.643921,32415.648735,14791.403745,3062.000000,167.000000,7489.250000,1234.000000
50%,17.725990,75.000000,1.180372,36923.000000,19320.000000,4163.000000,237.000000,10517.000000,1638.000000
75%,18.593661,88.000000,1.499169,49651.000000,21211.000000,5105.250000,268.000000,11129.750000,1841.000000
max,27.156440,98.000000,2.510210,66460.000000,26031.000000,6009.000000,526.995046,14958.000000,2959.000000
